In [1]:
import csv
import itertools

In [2]:
with open('C:\\Users\\m5in\\Documents\\FAC\\Marubozu\\NIFTY_SEP.txt', 'r') as in_file:
    stripped = (line.strip() for line in in_file)
    lines = (line.split(",") for line in stripped if line)
    with open('C:\\Users\\m5in\\Documents\\FAC\\Marubozu\\NIFTY_SEP.csv', 'w') as out_file:
        writer = csv.writer(out_file)
        writer.writerow(('Stock_Market','Trading_Day', 'Trading_Time', 'Open', 'High', 'Low', 'Close','#','*'))
        writer.writerows(lines)

In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("C:\\Users\\m5in\\Documents\\FAC\\Marubozu\\NIFTY_SEP.csv")
df

,Stock_Market,Trading_Day,Trading_Time,Open,High,Low,Close,#,*
0,NIFTY,20200901,09:08,11464.30,11464.30,11464.30,11464.30,0,0
1,NIFTY,20200901,09:16,11464.85,11464.85,11376.45,11394.30,0,0
2,NIFTY,20200901,09:17,11395.35,11402.80,11380.75,11402.80,0,0
3,NIFTY,20200901,09:18,11403.50,11408.55,11390.25,11393.05,0,0
4,NIFTY,20200901,09:19,11389.85,11389.85,11370.10,11380.80,0,0
...,...,...,...,...,...,...,...,...,...
8310,NIFTY,20200930,15:28,11224.35,11225.40,11222.40,11223.00,0,0
8311,NIFTY,20200930,15:29,11223.55,11223.70,11221.50,11223.10,0,0
8312,NIFTY,20200930,15:30,11222.90,11228.55,11222.10,11226.30,0,0
8313,NIFTY,20200930,15:31,11226.50,11226.50,11226.50,11226.50,0,0


In [5]:
df1 = {
    'Trading_Day':[None],
    'Req_Trading_Day':[None],
    'Trading_Time':[None],
    '09:30':[None],
    '10:00':[None],
    '10:30':[None],
    '11:00':[None],
    '11:30':[None],
    '12:00':[None],
    '12:30':[None],
    '13:00':[None],
    '13:30':[None],
    '14:00':[None],
    '14:30':[None],
    '15:00':[None],
    '15:30':[None],
    'Open':[None],
    'High':[None],
    'Low':[None],
    'Close':[None]
}

In [6]:
df1['Trading_Day'] = df.Trading_Day.unique()
arr = df1['Trading_Day']
df1["Trading_Day"] = pd.DataFrame(arr, columns = None)

In [7]:
x = len(df1['Trading_Day'])
arr = []
for i in range(x):
    for j in range(13):
        arr.append(df1['Trading_Day'].iloc[i])
arr = np.array(arr)
df1["Req_Trading_Day"] = pd.DataFrame(arr, columns =None)

In [8]:
time = ['09:30','10:00','10:30','11:00','11:30','12:00','12:30','13:00','13:30','14:00','14:30','15:00','15:30']

for i in range(len(time)):
    arr = df[df["Trading_Time"]==time[i]].index.values
    df1[time[i]] = pd.DataFrame(arr, columns = None)

In [9]:
time = ['09:30','10:00','10:30','11:00','11:30','12:00','12:30','13:00','13:30','14:00','14:30','15:00','15:30']
U_high = []
U_low = []
U_open = []
U_close = []
for i in range(len(df1["Trading_Day"])):
    #print(i)
    for j in range(0,len(time)-1):
        #print(time[j])
        High = 0
        Low = 1000000
        flag = 0
        if(j==0):
            for k in range(0,(df1[time[j]].iloc[i][0])):
                if(k==0):
                    U_open.append(df['Open'][0])
                if(k==df1[time[j]].iloc[i][0]-1):
                    U_close.append(df['Close'][k])
                if(df['High'][k]>High):
                    High = df['High'][k]
                elif(df['Low'][k]<Low):
                    Low = df['Low'][k]
                else:
                    continue
            U_high.append(High)
            U_low.append(Low)
            
        for k in range((df1[time[j]].iloc[i][0]),(df1[time[j+1]].iloc[i][0])):
            if(k==df1[time[j]].iloc[i][0]):
                U_open.append(df['Open'][k])
            if(k==df1[time[j+1]].iloc[i][0]-1):
                U_close.append(df['Close'][k])
            if(df['High'][k]>High):
                High = df['High'][k]
            elif(df['Low'][k]<Low):
                Low = df['Low'][k]
            else:
                continue
            #print(High,j)
        U_high.append(High)
        U_low.append(Low)

#print(U_open)

In [10]:
U_low = np.array(U_low)
U_high = np.array(U_high)
U_open = np.array(U_open)
U_close = np.array(U_close)

In [11]:
df1["High"] = pd.DataFrame(U_low , columns =None)
df1["Low"] = pd.DataFrame(U_high, columns = None)
df1["Open"] = pd.DataFrame(U_open, columns = None)
df1["Close"] = pd.DataFrame(U_close, columns = None)

In [12]:
#Bullish Marubozu
#Open ~ Low // Close ~ High
#Bearish Marubozu
#Open ~ High // Close~low

flag = 0
o_amount = 100000
amount = 100000
stock = 0
stoploss =0
for i in range(0,len(df1['Low'])):
    Open = df1['Open'].iloc[i]
    Open = Open.to_numpy()
    Low = df1['Low'].iloc[i]
    Low = Low.to_numpy()
    Close = df1['Close'].iloc[i]
    Close = Close.to_numpy()
    High = df1['High'].iloc[i]
    High = High.to_numpy()
    diff = abs(((Open[0]-Low[0])/(Open[0]))*100)
    diff1 = abs(((Close[0]-High[0])/(Close[0]))*100)
    tiff = abs(((Open[0]-High[0])/(High[0]))*100)
    tiff1 = abs(((Close[0]-Low[0])/(Low[0]))*100)
    
    if(diff<0.5 and diff1<0.5 and flag==0):
        Day = df1['Req_Trading_Day'].iloc[i]
        Day = Day.to_numpy()
        #print('Bullish Marubozu pattern found on:',Day[0])
        stock = amount/Close[0]
        amount = 0
        flag = 1
        stoploss = Low[0]
    if(tiff<0.5 and tiff1<0.5 and flag==1):
        Day = df1['Req_Trading_Day'].iloc[i]
        Day = Day.to_numpy()
        #print('Bearish Marubozu pattern found on:',Day[0])
        if(Close[0]<stoploss):
            #print(stoploss)
            amount += stock * Close[0]
            flag = 0
            stock =0
#loss = ((o_amount-amount)/(o_amount))*100
print(amount,stock)

100649.2595135811 0
